<a href="https://colab.research.google.com/github/purvanshh/plant-disease-decision-pipeline/blob/main/Binary_Classification_MobileNetV3_Small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
vipoooool_new_plant_diseases_dataset_path = kagglehub.dataset_download('vipoooool/new-plant-diseases-dataset')
arjuntejaswi_plant_village_path = kagglehub.dataset_download('arjuntejaswi/plant-village')

print('Data source import complete.')


Using Colab cache for faster access to the 'new-plant-diseases-dataset' dataset.
Using Colab cache for faster access to the 'plant-village' dataset.
Data source import complete.


In [ ]:
"""
TASK 2 - PLANT DISEASE CLASSIFIER
==================================
Binary classification of healthy vs diseased crops using MobileNetV3.
Dataset: PlantVillage dataset (≤5,000 images)
Hardware: Laptop CPU/GPU compatible
"""

import os
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import confusion_matrix
from pathlib import Path
from PIL import Image

# ========== CONFIG ==========
SEED = 42
MAX_IMAGES = 5000
MAX_PER_CLASS = 1100
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 15

# Dataset path from the original code
DATA_ROOT = Path("/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train")

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# ========== DATA LOADING ==========
def load_balanced_dataset():
    """Load and balance dataset from PlantVillage"""

    # Find all image files
    image_paths = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
        image_paths.extend(list(DATA_ROOT.rglob(ext)))

    if not image_paths:
        raise FileNotFoundError(f"No images found at {DATA_ROOT}")

    # Separate healthy and diseased
    healthy_paths = [p for p in image_paths if 'healthy' in p.parent.name.lower()]
    diseased_paths = [p for p in image_paths if p not in healthy_paths]

    print(f"Found {len(healthy_paths)} healthy images")
    print(f"Found {len(diseased_paths)} diseased images")

    # Apply limits
    random.shuffle(healthy_paths)
    random.shuffle(diseased_paths)

    healthy_sample = healthy_paths[:min(MAX_PER_CLASS, len(healthy_paths))]
    diseased_sample = diseased_paths[:min(MAX_PER_CLASS, len(diseased_paths))]

    # Combine and balance
    all_paths = healthy_sample + diseased_sample
    all_labels = [0] * len(healthy_sample) + [1] * len(diseased_sample)

    # Shuffle
    combined = list(zip(all_paths, all_labels))
    random.shuffle(combined)
    all_paths, all_labels = zip(*combined)

    # Apply final limit
    all_paths = list(all_paths)[:MAX_IMAGES]
    all_labels = list(all_labels)[:MAX_IMAGES]

    print(f"Final dataset: {len(all_paths)} images")
    print(f"Healthy: {sum(1 for l in all_labels if l == 0)}, Diseased: {sum(1 for l in all_labels if l == 1)}")

    return all_paths, all_labels

class PlantDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        try:
            img = Image.open(self.paths[idx]).convert('RGB')
            if self.transform:
                img = self.transform(img)
            return img, self.labels[idx]
        except Exception as e:
            # Return placeholder if image fails to load
            print(f"Error loading {self.paths[idx]}: {e}")
            return torch.zeros(3, IMAGE_SIZE, IMAGE_SIZE), self.labels[idx]

# Preprocessing transforms
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ========== MODEL ==========
class DiseaseClassifier(nn.Module):
    """MobileNetV3 for binary classification"""
    def __init__(self):
        super().__init__()
        self.model = models.mobilenet_v3_small(weights='IMAGENET1K_V1')

        # Freeze early layers
        for param in list(self.model.parameters())[:-10]:
            param.requires_grad = False

        # Replace classifier for binary classification
        in_features = self.model.classifier[3].in_features
        self.model.classifier[3] = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(in_features, 2)
        )

    def forward(self, x):
        return self.model(x)

# ========== TRAINING ==========
def train(model, train_loader, val_loader, epochs=15):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

    best_val_f1 = 0
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()
            train_total += images.size(0)

        # Validation
        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.numpy())

        # Calculate metrics
        cm = confusion_matrix(val_labels, val_preds)
        tn, fp, fn, tp = cm.ravel()
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

        train_acc = train_correct / train_total if train_total > 0 else 0
        train_loss = train_loss / train_total if train_total > 0 else 0

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.3f}, Train Acc={train_acc:.3f}, "
              f"Val Acc={accuracy:.3f}, Val F1={f1:.3f}")

        if f1 > best_val_f1:
            best_val_f1 = f1
            torch.save(model.state_dict(), 'best_model.pt')

    return model

# ========== EVALUATION ==========
def evaluate(model, test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()

    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    cm = confusion_matrix(all_labels, all_preds)
    tn, fp, fn, tp = cm.ravel()

    metrics = {
        'accuracy': (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0,
        'precision': tp / (tp + fp) if (tp + fp) > 0 else 0,
        'recall': tp / (tp + fn) if (tp + fn) > 0 else 0,
        'false_negative_rate': fn / (tp + fn) if (tp + fn) > 0 else 0,
        'false_positive_rate': fp / (tn + fp) if (tn + fp) > 0 else 0,
        'confusion_matrix': cm
    }

    print("\n" + "="*60)
    print("RESULTS")
    print("="*60)
    print(f"Accuracy: {metrics['accuracy']:.2%}")
    print(f"Precision: {metrics['precision']:.2%}")
    print(f"Recall (Critical): {metrics['recall']:.2%}")
    print(f"False Negative Rate: {metrics['false_negative_rate']:.2%}")
    print(f"False Positive Rate: {metrics['false_positive_rate']:.2%}")
    print(f"Confusion Matrix:\n{metrics['confusion_matrix']}")

    return metrics

# ========== FAILURE ANALYSIS ==========
def analyze_failures(model, test_dataset, num_cases=2):
    """Show examples where model was wrong"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()

    failures = []
    with torch.no_grad():
        for i in range(min(100, len(test_dataset))):
            img, true_label = test_dataset[i]
            img = img.unsqueeze(0).to(device)
            output = model(img)
            pred_label = torch.argmax(output).item()

            if pred_label != true_label:
                failures.append({
                    'index': i,
                    'true': 'Healthy' if true_label == 0 else 'Diseased',
                    'pred': 'Healthy' if pred_label == 0 else 'Diseased',
                    'path': test_dataset.paths[i]
                })
                if len(failures) >= num_cases:
                    break

    print("\n" + "="*60)
    print("FAILURE CASE ANALYSIS")
    print("="*60)

    for f in failures:
        print(f"\nCase: {f['path'].name}")
        print(f"Folder: {f['path'].parent.name}")
        print(f"True: {f['true']}, Predicted: {f['pred']}")
        if f['true'] == 'Diseased' and f['pred'] == 'Healthy':
            print("IMPACT: Missed disease - could lead to crop loss")
        else:
            print("IMPACT: False alarm - wasted treatment")

# ========== MAIN ==========
def main():
    print("="*60)
    print("TASK 2: PLANT DISEASE CLASSIFICATION")
    print("="*60)

    # 1. Load data
    print("\nLoading data...")
    paths, labels = load_balanced_dataset()

    # Split (70/15/15)
    from sklearn.model_selection import train_test_split
    train_paths, temp_paths, train_labels, temp_labels = train_test_split(
        paths, labels, test_size=0.3, stratify=labels, random_state=SEED
    )
    val_paths, test_paths, val_labels, test_labels = train_test_split(
        temp_paths, temp_labels, test_size=0.5, stratify=temp_labels, random_state=SEED
    )

    # Create datasets
    train_dataset = PlantDataset(train_paths, train_labels, train_transform)
    val_dataset = PlantDataset(val_paths, val_labels, val_transform)
    test_dataset = PlantDataset(test_paths, test_labels, val_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

    print(f"Train: {len(train_dataset)} images")
    print(f"Val: {len(val_dataset)} images")
    print(f"Test: {len(test_dataset)} images")

    # 2. Initialize model
    print("\nInitializing model...")
    model = DiseaseClassifier()
    print(f"Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
    print(f"Total params: {sum(p.numel() for p in model.parameters()):,}")

    # 3. Train
    print("\nTraining...")
    model = train(model, train_loader, val_loader, EPOCHS)

    # 4. Load best model and evaluate
    print("\nLoading best model...")
    model.load_state_dict(torch.load('best_model.pt'))
    metrics = evaluate(model, test_loader)

    # 5. Failure analysis
    analyze_failures(model, test_dataset)

    # 6. Save final model
    torch.save(model.state_dict(), 'plant_disease_model.pt')
    model_size_mb = os.path.getsize('plant_disease_model.pt') / (1024 * 1024)
    print(f"\nModel saved (size: {model_size_mb:.1f} MB)")

    # 7. Simple report
    print("\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    print(f"Dataset: {len(paths)} images (≤{MAX_IMAGES})")
    print(f"Model: MobileNetV3-small")
    print(f"Accuracy: {metrics['accuracy']:.2%}")
    print(f"Recall: {metrics['recall']:.2%} (critical for disease detection)")
    print(f"False Negative Rate: {metrics['false_negative_rate']:.2%}")
    print(f"Model Size: {model_size_mb:.1f} MB")
    print(f"Requirements Met: ≤5,000 images, laptop feasible, offline capable")

if __name__ == "__main__":
    main()

TASK 2: PLANT DISEASE CLASSIFICATION

Loading data...
Found 22294 healthy images
Found 48001 diseased images
Final dataset: 2200 images
Healthy: 1100, Diseased: 1100
Train: 1540 images
Val: 330 images
Test: 330 images

Initializing model...
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|██████████| 9.83M/9.83M [00:00<00:00, 90.5MB/s]


Trainable params: 704,834
Total params: 1,519,906

Training...
Epoch 1: Train Loss=0.375, Train Acc=0.830, Val Acc=0.685, Val F1=0.540
Epoch 2: Train Loss=0.196, Train Acc=0.933, Val Acc=0.867, Val F1=0.846
Epoch 3: Train Loss=0.142, Train Acc=0.953, Val Acc=0.912, Val F1=0.904
Epoch 4: Train Loss=0.123, Train Acc=0.956, Val Acc=0.909, Val F1=0.901
Epoch 5: Train Loss=0.106, Train Acc=0.956, Val Acc=0.955, Val F1=0.953
Epoch 6: Train Loss=0.096, Train Acc=0.964, Val Acc=0.970, Val F1=0.970
Epoch 7: Train Loss=0.087, Train Acc=0.971, Val Acc=0.961, Val F1=0.960
Epoch 8: Train Loss=0.069, Train Acc=0.973, Val Acc=0.964, Val F1=0.964
Epoch 9: Train Loss=0.066, Train Acc=0.976, Val Acc=0.948, Val F1=0.947
Epoch 10: Train Loss=0.066, Train Acc=0.976, Val Acc=0.958, Val F1=0.957
Epoch 11: Train Loss=0.039, Train Acc=0.990, Val Acc=0.967, Val F1=0.966
Epoch 12: Train Loss=0.055, Train Acc=0.982, Val Acc=0.973, Val F1=0.973
Epoch 13: Train Loss=0.034, Train Acc=0.988, Val Acc=0.979, Val F1=0.9

In [ ]:
"""
TASK 3: FROM PREDICTION TO DECISION (Simplified)
==================================================
Converts model outputs (12% diseased, 70% confidence) into farmer recommendations.
Focuses on core decision logic without unnecessary complexity.
"""

def make_decision(percentage_diseased=12.0, confidence=0.70, budget="limited", days_to_harvest=30):
    """
    Simple decision logic for farmers.

    Args:
        percentage_diseased: % of field flagged (default: 12%)
        confidence: model confidence (default: 0.70)
        budget: "limited", "moderate", or "ample"
        days_to_harvest: days until harvest (default: 30)

    Returns: recommendation dict
    """

    # ========== 1. WHY CONFIDENCE IS INSUFFICIENT ==========
    """
    Reason: 70% confidence doesn't mean 70% accuracy.
    Example: If model is poorly calibrated, 70% confidence could be:
    - Actually 90% accurate (underconfident)
    - Actually 50% accurate (overconfident)
    - Confidence doesn't consider field conditions, disease type, or weather
    """

    # ========== 2. ASSESS RISK ==========
    if percentage_diseased < 5:
        risk = "low"
    elif percentage_diseased < 20:
        risk = "moderate"
    elif percentage_diseased < 40:
        risk = "high"
    else:
        risk = "critical"

    # ========== 3. CONSIDER CONFIDENCE ==========
    if confidence < 0.60:
        confidence_level = "low"
        verification_needed = True
    elif confidence < 0.75:
        confidence_level = "moderate"
        verification_needed = True
    elif confidence < 0.90:
        confidence_level = "high"
        verification_needed = False
    else:
        confidence_level = "very high"
        verification_needed = False

    # ========== 4. APPLY BUDGET CONSTRAINT ==========
    budget_constraints = {
        "limited": {"can_verify": True, "can_treat_partial": True, "can_treat_full": False},
        "moderate": {"can_verify": True, "can_treat_partial": True, "can_treat_full": False},
        "ample": {"can_verify": True, "can_treat_partial": True, "can_treat_full": True}
    }
    budget_info = budget_constraints[budget]

    # ========== 5. CHECK TIME CONSTRAINT ==========
    if days_to_harvest < 7:
        # Too late for treatment
        action = "harvest_early"
        reason = "Too close to harvest for effective treatment. Harvest now to save what you can."
    else:
        # ========== 6. DECISION LOGIC ==========
        if risk == "low":
            action = "monitor_only"
            reason = f"Only {percentage_diseased:.1f}% infection. Below action threshold."

        elif risk == "moderate":
            if verification_needed and budget_info["can_verify"]:
                action = "verify_then_treat"
                reason = f"{percentage_diseased:.1f}% infection with {confidence:.0%} confidence. Verify first to avoid unnecessary treatment."
            elif budget_info["can_treat_partial"]:
                action = "treat_flagged_areas"
                reason = f"Treat {percentage_diseased:.1f}% of field. Costs less than potential crop loss."
            else:
                action = "treat_high_confidence_areas"
                reason = "Limited budget. Treat only areas with highest confidence."

        elif risk == "high":
            if budget_info["can_treat_partial"]:
                action = "treat_flagged_areas_with_buffer"
                reason = f"High risk ({percentage_diseased:.1f}% infection). Treat flagged areas plus 1m buffer."
            else:
                action = "treat_most_critical"
                reason = "Limited budget. Treat only most critical infected areas first."

        else:  # critical
            if budget_info["can_treat_full"]:
                action = "treat_whole_field"
                reason = "Critical infection. Treat entire field to prevent total loss."
            else:
                action = "emergency_partial_treatment"
                reason = "Critical infection but limited budget. Treat as much as possible, prioritize containment."

    # ========== 7. COST ANALYSIS ==========
    """
    False Negative (miss disease) vs False Positive (false alarm) impact:
    - FN cost: ~$200 per plant (crop loss, spread to others)
    - FP cost: ~$15 per plant (wasted treatment)
    - Ratio: 13:1 → Missing disease is 13× more costly than false alarm
    """
    fn_cost = 200  # per plant
    fp_cost = 15   # per plant

    # ========== 8. ADDITIONAL SIGNALS THAT WOULD HELP ==========
    additional_signals = [
        "Weather forecast (rain/humidity increases disease spread)",
        "Disease type (fungal vs bacterial vs viral)",
        "Spatial pattern (clustered vs scattered)",
        "Crop growth stage",
        "Historical field performance",
        "Neighboring field reports"
    ]

    # ========== 9. RETURN RECOMMENDATION ==========
    return {
        "input": {
            "percentage_diseased": percentage_diseased,
            "confidence": confidence,
            "budget": budget,
            "days_to_harvest": days_to_harvest
        },
        "assessment": {
            "risk_level": risk,
            "confidence_level": confidence_level,
            "verification_needed": verification_needed,
            "budget_constraints": budget_info
        },
        "recommendation": {
            "action": action,
            "reason": reason,
            "next_steps": get_next_steps(action, percentage_diseased)
        },
        "cost_analysis": {
            "false_negative_cost_per_plant": fn_cost,
            "false_positive_cost_per_plant": fp_cost,
            "cost_ratio": fn_cost / fp_cost,
            "implication": "System should be biased toward catching diseases (accept more false alarms)"
        },
        "additional_signals": additional_signals,
        "confidence_limitation": "Model confidence alone is insufficient because it doesn't account for field conditions, disease type, or weather patterns that affect disease spread and treatment effectiveness."
    }

def get_next_steps(action, percentage_diseased):
    """Get concrete next steps for the farmer"""
    steps_map = {
        "monitor_only": [
            f"Rescan field in 7-10 days",
            "Monitor weather conditions",
            "Check for disease spread"
        ],
        "verify_then_treat": [
            f"Visit {percentage_diseased:.1f}% of field (flagged areas)",
            "Take 20-30 physical samples for verification",
            "Treat only confirmed diseased areas",
            "Monitor effectiveness in 5-7 days"
        ],
        "treat_flagged_areas": [
            f"Treat {percentage_diseased:.1f}% of field area",
            "Add buffer zone around treated areas",
            "Use appropriate fungicide",
            "Monitor for spread"
        ],
        "treat_high_confidence_areas": [
            f"Treat top {min(50, percentage_diseased * 2):.1f}% of flagged areas",
            "Focus on clustered areas first",
            "Treat perimeter to contain spread",
            "Plan follow-up if budget allows"
        ],
        "treat_flagged_areas_with_buffer": [
            f"Treat {percentage_diseased:.1f}% area plus 1m buffer",
            "Consider preventive treatment on edges",
            "Monitor closely for 2 weeks"
        ],
        "treat_most_critical": [
            "Treat areas with highest confidence scores",
            "Treat clustered infection centers",
            "Contain spread with perimeter treatment"
        ],
        "treat_whole_field": [
            "Treat entire field immediately",
            "Notify neighboring farms",
            "Monitor for 2 weeks"
        ],
        "emergency_partial_treatment": [
            "Treat as much as budget allows",
            "Focus on containment",
            "Contact agricultural extension for assistance"
        ],
        "harvest_early": [
            "Harvest immediately",
            "Separate diseased produce",
            "Clean equipment thoroughly"
        ]
    }
    return steps_map.get(action, ["Monitor field regularly"])

def print_recommendation(recommendation):
    """Print formatted recommendation"""
    print("\n" + "="*60)
    print("AGRICULTURAL DECISION SYSTEM")
    print("="*60)

    print(f"\n📊 SITUATION:")
    print(f"  • Field infection: {recommendation['input']['percentage_diseased']:.1f}%")
    print(f"  • Model confidence: {recommendation['input']['confidence']:.0%}")
    print(f"  • Farmer budget: {recommendation['input']['budget']}")
    print(f"  • Days to harvest: {recommendation['input']['days_to_harvest']}")

    print(f"\n🔍 ASSESSMENT:")
    print(f"  • Risk level: {recommendation['assessment']['risk_level'].upper()}")
    print(f"  • Confidence: {recommendation['assessment']['confidence_level'].upper()}")
    print(f"  • Verification needed: {recommendation['assessment']['verification_needed']}")

    print(f"\n🎯 RECOMMENDATION: {recommendation['recommendation']['action'].replace('_', ' ').title()}")
    print(f"  Reason: {recommendation['recommendation']['reason']}")

    print(f"\n📋 NEXT STEPS:")
    for i, step in enumerate(recommendation['recommendation']['next_steps'], 1):
        print(f"  {i}. {step}")

    print(f"\n💰 COST ANALYSIS:")
    print(f"  • Missed disease cost: ${recommendation['cost_analysis']['false_negative_cost_per_plant']}/plant")
    print(f"  • False alarm cost: ${recommendation['cost_analysis']['false_positive_cost_per_plant']}/plant")
    print(f"  • Cost ratio: {recommendation['cost_analysis']['cost_ratio']:.0f}:1")
    print(f"  • Implication: {recommendation['cost_analysis']['implication']}")

    print(f"\n⚠️  CONFIDENCE LIMITATION:")
    print(f"  {recommendation['confidence_limitation']}")

    print(f"\n📡 ADDITIONAL SIGNALS NEEDED:")
    for i, signal in enumerate(recommendation['additional_signals'][:3], 1):
        print(f"  {i}. {signal}")

def test_scenarios():
    """Test different scenarios"""
    print("\n" + "="*60)
    print("SCENARIO TESTING")
    print("="*60)

    scenarios = [
        {"name": "Base scenario (12%, 70%, limited)",
         "args": {"percentage_diseased": 12.0, "confidence": 0.70, "budget": "limited"}},
        {"name": "Low confidence (12%, 50%, limited)",
         "args": {"percentage_diseased": 12.0, "confidence": 0.50, "budget": "limited"}},
        {"name": "High risk (25%, 70%, limited)",
         "args": {"percentage_diseased": 25.0, "confidence": 0.70, "budget": "limited"}},
        {"name": "Ample budget (12%, 70%, ample)",
         "args": {"percentage_diseased": 12.0, "confidence": 0.70, "budget": "ample"}},
        {"name": "Critical (45%, 70%, limited)",
         "args": {"percentage_diseased": 45.0, "confidence": 0.70, "budget": "limited"}},
    ]

    for scenario in scenarios:
        print(f"\n📝 {scenario['name']}:")
        rec = make_decision(**scenario['args'])
        print(f"  → Action: {rec['recommendation']['action'].replace('_', ' ').title()}")

def main():
    """Main function"""
    print("="*60)
    print("TASK 3: FROM PREDICTION TO DECISION")
    print("="*60)

    # Default scenario from problem statement
    print("\nDefault scenario: 12% diseased, 70% confidence, limited budget")
    recommendation = make_decision()
    print_recommendation(recommendation)

    # Test other scenarios
    test_scenarios()

    print("\n" + "="*60)
    print("KEY INSIGHTS")
    print("="*60)
    print("1. Confidence ≠ accuracy - need verification for moderate confidence")
    print("2. False negatives (missed disease) cost 13× more than false positives")
    print("3. Limited budget → targeted verification/treatment")
    print("4. Near harvest → early harvest may be best option")
    print("5. Always consider: What additional info would change the decision?")

if __name__ == "__main__":
    main()

TASK 3: FROM PREDICTION TO DECISION

Default scenario: 12% diseased, 70% confidence, limited budget

AGRICULTURAL DECISION SYSTEM

📊 SITUATION:
  • Field infection: 12.0%
  • Model confidence: 70%
  • Farmer budget: limited
  • Days to harvest: 30

🔍 ASSESSMENT:
  • Risk level: MODERATE
  • Confidence: MODERATE
  • Verification needed: True

🎯 RECOMMENDATION: Verify Then Treat
  Reason: 12.0% infection with 70% confidence. Verify first to avoid unnecessary treatment.

📋 NEXT STEPS:
  1. Visit 12.0% of field (flagged areas)
  2. Take 20-30 physical samples for verification
  3. Treat only confirmed diseased areas
  4. Monitor effectiveness in 5-7 days

💰 COST ANALYSIS:
  • Missed disease cost: $200/plant
  • False alarm cost: $15/plant
  • Cost ratio: 13:1
  • Implication: System should be biased toward catching diseases (accept more false alarms)

⚠️  CONFIDENCE LIMITATION:
  Model confidence alone is insufficient because it doesn't account for field conditions, disease type, or weathe